In [1]:
import requests
oobabooga_api_host = "127.0.0.1"
oobabooga_api_port = "5000"
main_lm_temperature = 0.8
oobabooga_api_server = f'http://{oobabooga_api_host}:{oobabooga_api_port}'

In [53]:
def api_encode_text(text: str):
	json = {
		"text": text,
	}
	response = requests.post(url=f"{oobabooga_api_server}/v1/internal/encode", json=json).json()
	return response["tokens"]

def api_decode_tokens(tokens: list):
	json = {
		"tokens": tokens,
	}
	response = requests.post(url=f"{oobabooga_api_server}/v1/internal/decode", json=json).json()
	print(json)
	print(response)
	return response["text"]
	

def api_generate_response(prompt: str,
						  temperature: float = 0.7,
						  max_tokens: int = 200,
						  top_k: int = 20,
						  top_p: float = 1.0,
						  logit_bias: dict = {},
						  stop: list = ["\n"],):
	json = {
		"prompt": prompt,
		"temperature": temperature,
		"max_tokens": max_tokens,
		"top_k": top_k,
		"top_p": top_p,
		"logit_bias": logit_bias, # e.g. {"personality": 2.0}
		"stop": stop,
	}
	return ((requests.post(url=f"{oobabooga_api_server}/v1/completions", json=json)).json())['choices'][0]['text']


In [24]:
def api_select(prompt, options, temperature, top_k, top_p):
    tokenized_options = [api_encode_text(option) for option in options]
    while len(tokenized_options) != 1:
        all_first_tokens = [option[0] for option in tokenized_options]
        tokens_to_options = {}
        for i in range(len(all_first_tokens)):
            if all_first_tokens[i] not in tokens_to_options.keys():
                tokens_to_options[all_first_tokens[i]] = [tokenized_options[i]]
            else:
                tokens_to_options[all_first_tokens[i]].append(tokenized_options[i])

        logit_bias = {}
        for tokens_for_check in tokens_to_options.keys():
            logit_bias[tokens_for_check] = 100

        response = api_generate_response(prompt=prompt, temperature=temperature, top_k=top_k, top_p=top_p, logit_bias=logit_bias, max_tokens=2)
        response_token = api_encode_text(response)[0]
        tokenized_options = tokens_to_options[response_token]

    return tokenized_options[0]


In [122]:
def api_select(prompt, options, temperature, top_k, top_p):
    if not options:
        return None  # Handle the case of empty options list
    options = [option.strip() for option in options]
    tokenized_options = [api_encode_text(option) for option in options]
    answer = ""
    round_number = 1
    while len(tokenized_options) > 1:  # Use > instead of != to ensure termination
        round_number += 1
        all_first_tokens = [option[0] for option in tokenized_options]
        tokens_to_options = {}
        for i in range(len(all_first_tokens)):
            if all_first_tokens[i] not in tokens_to_options:
                tokens_to_options[all_first_tokens[i]] = [tokenized_options[i]]
            else:
                tokens_to_options[all_first_tokens[i]].append(tokenized_options[i])
        logit_bias = {}
        for tokens_for_check in tokens_to_options:
            logit_bias[tokens_for_check] = 100

        response = api_generate_response(prompt=prompt, temperature=temperature, top_k=top_k, top_p=top_p, logit_bias=logit_bias, max_tokens=1)
        response_token = api_encode_text(response)
        response_token = response_token[-1]
        prompt += response
        answer += response
        if response_token in tokens_to_options.keys():
            for i in tokens_to_options[response_token]:
                if len(i) == 0:
                    break
            tokenized_options = [i[1:] for i in tokens_to_options[response_token]]

            
        else:
            # Handle the case where the response token is not found among the options
            break  # Exit the loop to avoid potential infinite loop
        # decode the final tokenized answer
    for i in options:
        if i.startswith(answer.strip()):
            answer = i
            break
    return answer


In [126]:
print(api_select("What is the best programming language?", ["Python", "Java", "C++", "JavaScript"], main_lm_temperature, 20, 1.0))

C++


In [107]:
api_decode_tokens([1, 28705])

{'tokens': [1, 28705]}
{'text': ''}


''

In [83]:
api_decode_tokens(api_encode_text("hello world"))

{'tokens': [1, 6312, 28709, 1526]}
{'text': 'hello world'}


'hello world'

In [129]:

# Longer prompts with characters making decisions
prompt = "You are in the middle of a dense forest. It's getting dark, and you realize you've lost your way. You come across a fork in the path. The left path seems to lead deeper into the forest, while the right path appears to go uphill. Which way do you choose?"
options = ["Take the left path", "Take the right path", "Try to backtrack"]

temperature = 0.7
top_k = 5
top_p = 0.9

# Test the function
selected_option = api_select(prompt, options, temperature, top_k, top_p)
print("Selected option:", selected_option)

Selected option: Take the left path


In [ ]:
AI_Council_data = [{
    "name": "Lumi",
    "personality_prompt": "This is Lumi: Objective, logical, cold, analytical. Aware of patterns, aware of trends. She's efficient, and a prick"
},
{
    "name": "Reverie",
    "personality_prompt": "This is Reverie: Subjective, creative, sensory. Aware of feelings, aware of people. She's emotional, and an idiot"
}]

def council_of_thought():
	global AI_Council_data
 
	# Generate prompt for each council member
	council_member_prompt = ""
	for member in AI_Council_data:
		council_member_prompt += f"[{member['name']}]\n- {member['personality_prompt']}\n"
  
	council_prompt = f"""\
The following are Lucid's council members. Each member has a unique perspective and role in Lucid's decision-making process.
{council_member_prompt.strip()}

In conversations, the input from the outside world will be included, with text inside parentheses ( ) to denote thoughts within Lucid's head.
Below is an example conversation between Lucid and its council members, along with interactions with Miko.

[Example]
Lucid: Miko, it seems you're struggling with your code again.
Miko: Yeah, I can't seem to find the bug. It's driving me crazy.
(Lumi: Miko's frustration seems to be hindering his productivity.)
(Reverie: Maybe he needs a break to clear his mind.)
(Lucid: Okay, let's see if we can find the bug together.)
Lucid: Perhaps it's time for a short break, Miko. Clear your head, then we'll tackle this bug together.
[End of Example]
"""
	return council_prompt
print(council_of_thought())

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("Ichigo2899/MixTAO-7Bx2-MoE-v8.1-AWQ")
model = AutoModelForCausalLM.from_pretrained("Ichigo2899/MixTAO-7Bx2-MoE-v8.1-AWQ", attn_implementation="flash_attention_2", device_map="cuda:0")

my_config = GenerationConfig(tempture=0.7, 
                             max_new_tokens=100,
                             eos_token_id=model.config.eos_token_id,
                             pad_token=model.config.pad_token_id,
)

In [ ]:
model.config

In [ ]:
inputs = tokenizer("Once upon a time, there was a young", return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, generation_config=my_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import requests
import time
host = '127.0.0.1'
port = '8001'
server = f'http://{host}:{port}'
def send_output(output, server=server):
	requests.post(url=f"{server}/output",json=output)   

In [ ]:
response = {
        'source' : 'Lucid',
        'content' : 'generated_response',
        'timestamp' : time.time(),
        'type' : 'conversation',
        }
send_output(response)

In [ ]:
from guidance import models, gen, select
import guidance
lm = models.Transformers('solidrust/Hercules-4.0-Mistral-v0.2-7B-AWQ', device_map="cuda", echo=True, temperature=0.8)


In [ ]:
import requests
oobabooga_api_server = "http://127.0.0.1:5000"
def api_generate_response(prompt: str,
                          temperature: float,
                          max_tokens: int,
                          top_k: int,
                          top_p: float,
                          stop: list,):
	json = {
		"prompt": prompt,
		"temperature": temperature,
		"max_tokens": max_tokens,
		"top_k": top_k,
		"top_p": top_p,
		"stop": stop,
	}
	return ((requests.post(url=f"{oobabooga_api_server}/v1/completions", json=json)).json())['choices'][0]['text']

In [ ]:
my_object = api_generate_response("Once upon a time, there was a young", 0.7, 100, 50, 0.9, ["\n"])

In [ ]:
print(my_object.json())
print(my_object.json()['choices'][0]['text'])
print(my_object)

In [ ]:
@guidance(stateless=True)
def test(lm):
    lm += "The quick brown fox "+gen(stop=".", max_tokens=100, temperature=0.8)
    return lm

lm+test()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("oscar", "unshuffled_original_ta")

In [ ]:
from datetime import date

conversation = []

def get_conversation(conversation=conversation, retrieval_amount=8):
    if len(conversation) == 0:
        return 'No Record Yet.'
    else:
        prompt = ''
        if len(conversation) < retrieval_amount:
            for i in range(len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        else:
            start_index = max(len(conversation) - retrieval_amount, 0)
            for i in range(start_index, len(conversation)):
                prompt += f"[{date.fromtimestamp(conversation[i]['timestamp'])}] {conversation[i]['source']}: {conversation[i]['content']}\n"
        return prompt.strip()

# Example usage:
conversation = [
    {'source': 'user', 'content': 'Hello!', 'timestamp': 1647289200},
    {'source': 'assistant', 'content': 'Hi there!', 'timestamp': 1647292800},
    {'source': 'user', 'content': 'How are you?', 'timestamp': 1647296400},
    {'source': 'assistant', 'content': "I'm doing well, thank you!", 'timestamp': 1647300000},
]

print(get_conversation(conversation))


In [ ]:
def yield_test(n):
    print("start n =", n)
    for i in range(n):
        yield i*i
        print("i =", i)
        if i == 2:  # Add a condition to return early
            return
    print("end")

tests = yield_test(5)
for test in tests:
    print("test =", test)
    print("--------")

In [ ]:
lm2 =lm+ "Hello! I "+gen(max_tokens=10, temperature=0.35)

In [ ]:

import chromadb
from sentence_transformers import SentenceTransformer
#sentences = ["This is an example sentence", "Each sentence is converted"]

sentence_transformer = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

import time
from datetime import date

In [ ]:

#embeddings = sentence_transformer.encode(sentences)#
short_term_memory = chromadb.Client()
#client = chromadb.PersistentClient(path="./Chroma")
working_memory = []
"""
This is what a Info Block should look like
{
	'object_type' : 'entity',
	'object_name' : 'Miko',
	'content' : 'Miko like Nintendo games.',
	'timestamp' : '2021-10-15 17:37:00',
	'vector' array([[-4.39221077e-02, -1.25277145e-02,  2.93133650e-02,]], dtype=float32): ,
}    

with guidance
f'''\
{
	'object_type' : '{select(['entity','events'])'},
	'object_name' : '{gen(stop='\'')},
	'content' : '{gen(stop='\'')},
	'timestamp' : {date.fromtimestamp(time.time())},
	'vector' : {sentence_transformer.encode([])},
}'''
"""
#def process_new_info_block(lm_text_result):

@guidance(stateless=True)
def guidance_make_new_info_block(lm, passage):
		first_curly = "{"
		second_curly = "}"
		lm += """\
This is what a Info Block should look like from an example:
[Example]
Lucid: What are you doing now? Stop ignoring me!
Miko: I'm playing Nintendo games.
Lucid: And that's a higher priority than me?
Miko: ...
Lucid: Your silence speaks volumes.

```json
{
	"object_type" : 'entity',
	"object_name" : 'Miko',
	"content" : 'Miko likes Nintendo games.',
}
```
[End of Example]
"""
		lm += f"""\
{passage}
```json
{first_curly}
	"object_type" : '{select(['entity','events'], name="object_type")},
	"object_name" : "{gen(stop='"',name = "object_name")},
	"content" : "{gen(stop='"',name="content")},"""  
		return lm

def make_new_info_block(lm, passage):
	first_curly = "{"
	second_curly = "}"
	lm += guidance_make_new_info_block(passage)
	print(lm['object_type'],lm['object_name'],lm['content'])
	timestamp = date.fromtimestamp(time.time())
	json_file ={
	"object_type" : lm['object_type'],
	"object_name" : lm['object_name'],
	"content" : lm['content'],
	"timestamp" : timestamp,
	"vector" : sentence_transformer.encode([lm['content']]),
	}
	return json_file

In [ ]:
passage = """\
Lucid: Hey Miko what's your favorite food?
Miko: Umm I don't know maybe rice?
Lucid you are really boring you know?"""
make_new_info_block(lm, passage)

In [ ]:
import sys
import traceback


class Item:
    SPACE = ' '
    DASH_SPACE = ' - '
    def __init__(self, key, name, function, parent=None):
        self.key = str(key)
        self.name = name
        self.function = function
        self.parent = parent

        if parent:
            parent.add_item(self)

    def __str__(self):
        return self.SPACE + self.key + self.DASH_SPACE + self.name

    def execute(self):
        print('\n' + self.name + '\n')
        return self.function()


class Menu:
    def __init__(self, name, items=None):
        self.name = name
        self.items = items or []

    def add_item(self, item):
        self.items.append(item)
        if item.parent != self:
            item.parent = self

    def remove_item(self, item):
        self.items.remove(item)
        if item.parent == self:
            item.parent = None

    def __str__(self):
        all = ['', self.name, '']
        for item in self.items:
            all.append(str(item))
        return '\n'.join(all)
            
    def get_item(self, key):
        for item in self.items:
            if item.key == key:
                return item

    def execute(self):
        while True:
            try:
                print(self)
                choice = raw_input(" >>  ")
                try:
                    item = self.get_item(choice)

                    try:
                        item.execute()
                    except Exception:
                        traceback.print_exc()

                except Exception:
                    print('Invalid selection, please try again.\n')
            except KeyboardInterrupt:
                sys.exit()
    
    
if __name__ == "__main__":
    # Pass in methods to run for each menu item
    menu_items = [
        Item(1, 'View Running tasks', view_running_tasks),
        Item(2, 'View Task', view_task),
        # ...
        Item(9, 'Exit', exit)]
    menu = Menu('Task Management', items=menu_items)

    menu.execute()